In [1]:
import random
import time
from tqdm import tqdm

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
MODEL_NAME = "google/flan-t5-small"
AG_LABELS = ["World", "Sports", "Business", "Sci/Tech"]
FEW_SHOT_K = 3
ONE_SHOT_K = 1
MAX_TEST_EXAMPLES = 500
GEN_MAX_LENGTH = 50  # модель вернёт короткий ответ (несколько токенов)
GEN_TEMPERATURE = 0.0  # детерминированный вывод
GEN_NUM_BEAMS = 1  # жадный поиск

# Пауза между запросами (чтобы не перегрузить cpu)
PAUSE_BETWEEN_CALLS = 0.1

In [3]:
print("Загружаем AG News из Hugging Face Datasets...")
dataset = load_dataset("ag_news")

train_ds = dataset["train"]
test_ds = dataset["test"]

print(f"Размер train: {len(train_ds)} статей")
print(f"Размер test : {len(test_ds)} статей")

Загружаем AG News из Hugging Face Datasets...
Размер train: 120000 статей
Размер test : 7600 статей


In [10]:
# Перемешаем train и выберем FEW_SHOT_K примеров для подсказок (они не будут участвовать в оценке)
indices = list(range(len(train_ds)))
random.shuffle(indices)

few_shot_indices = indices[:FEW_SHOT_K]
few_shot_examples = [train_ds[i] for i in few_shot_indices]

# В качестве one-shot возьмём первые ONE_SHOT_K из few-shot
one_shot_examples = few_shot_examples[:ONE_SHOT_K]

print(f"\nfew-shot ({FEW_SHOT_K} примеров) взято из train:")
for ex in few_shot_examples:
    lbl = AG_LABELS[ex["label"]]
    print(f"  • [{lbl}] {ex['text'][:60].strip()}...")

print(f"\none-shot (1 пример) — первый из few-shot: [{AG_LABELS[one_shot_examples[0]['label']]}] "
      f"{one_shot_examples[0]['text'][:60].strip()}...")
print()


few-shot (3 примеров) взято из train:
  • [Sci/Tech] List of 2004 MacArthur Foundation Fellows (AP) AP - The list...
  • [Business] MG Rover to slash directors #39; pension payments by 90 MG R...
  • [Sports] Hawk one-ups himself It turns out Richard Awa was just warmi...

one-shot (1 пример) — первый из few-shot: [Sci/Tech] List of 2004 MacArthur Foundation Fellows (AP) AP - The list...



In [11]:
print(f"Загружаем модель и токенизатор {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

device = torch.device("cpu")
model.to(device)
model.eval()
print("Модель загружена и переключена на CPU.\n")

Загружаем модель и токенизатор google/flan-t5-small...
Модель загружена и переключена на CPU.



In [12]:
def build_prompt_zero(text: str) -> str:  # для формирования zero-shot подсказки. Даем инструкцию + текст статьи и поясняем, что хотим получить метку
    instruction = (
        "Classify the following news article into one of the categories: "
        + ", ".join(AG_LABELS)
        + ".\n"
    )
    prompt = (
        instruction
        + "Article:\n\"\"\"\n"
        + text.strip()
        + "\n\"\"\"\n"
        + "Category:"
    )
    return prompt

def build_prompt_one(text: str, one_example: dict) -> str:  # для формирования one-shot подсказки, даем инструкцию + один пример + текст и просим метку
    instruction = (
        "Classify the following news article into one of the categories: "
        + ", ".join(AG_LABELS)
        + ".\n"
    )
    example_text = one_example["text"].strip()
    example_label = AG_LABELS[one_example["label"]]
    prompt = (
        instruction
        + "Example:\n"
        + "Article:\n\"\"\"\n"
        + example_text
        + "\n\"\"\"\n"
        + "Category: "
        + example_label
        + "\n---\n"
        + "Now classify this article:\n"
        + "Article:\n\"\"\"\n"
        + text.strip()
        + "\n\"\"\"\n"
        + "Category:"
    )
    return prompt

def build_prompt_few(text: str, few_examples: list) -> str:  # для формирования few-shot подсказки, даем инструкцию + несколько примеров + новый текст и просим метку
    instruction = (
        "Classify the following news article into one of the categories: "
        + ", ".join(AG_LABELS)
        + ".\n"
    )
    prompt = instruction
    for ex in few_examples:
        ex_text = ex["text"].strip()
        ex_label = AG_LABELS[ex["label"]]
        prompt += (
            "Example:\n"
            + "Article:\n\"\"\"\n"
            + ex_text
            + "\n\"\"\"\n"
            + "Category: "
            + ex_label
            + "\n---\n"
        )
    prompt += (
        "Now classify this article:\n"
        + "Article:\n\"\"\"\n"
        + text.strip()
        + "\n\"\"\"\n"
        + "Category:"
    )
    return prompt

In [13]:
def classify_text(prompt: str) -> str:
    # Токенизируем ввод
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512,
    ).to(device)

    # Генерируем ответ
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=GEN_MAX_LENGTH,
            temperature=GEN_TEMPERATURE,
            num_beams=GEN_NUM_BEAMS,
            early_stopping=True,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    for lbl in AG_LABELS:
        if lbl.lower() in decoded.lower():
            return lbl
    return decoded

In [14]:
def run_mode(mode: str):
    assert mode in ["zero", "one", "few"], "mode must be one of 'zero', 'one', 'few'"

    predictions = []
    true_labels = []

    # Если нужно ограничить число примеров для быстрой проверки
    total = len(test_ds) if MAX_TEST_EXAMPLES is None else min(len(test_ds), MAX_TEST_EXAMPLES)
    print(f"\nЗапуск режима {mode}-shot на {total} примерах из test...")

    for idx in tqdm(range(total), desc=f"{mode}-shot classification"):
        ex = test_ds[idx]
        text = ex["text"]
        true_labels.append(AG_LABELS[ex["label"]])

        if mode == "zero":
            prompt = build_prompt_zero(text)
        elif mode == "one":
            prompt = build_prompt_one(text, one_shot_examples[0])
        else:  # mode == "few"
            prompt = build_prompt_few(text, few_shot_examples)

        pred = classify_text(prompt)
        predictions.append(pred)

        time.sleep(PAUSE_BETWEEN_CALLS)

    return true_labels, predictions

In [15]:
def compute_accuracy(y_true: list, y_pred: list) -> float:
    correct = sum(1 for yt, yp in zip(y_true, y_pred) if yt.lower() == yp.lower())
    return correct / len(y_true) if len(y_true) > 0 else 0.0

In [16]:
# Zero-shot
y_true_zero, y_pred_zero = run_mode("zero")
acc_zero = compute_accuracy(y_true_zero, y_pred_zero)
print(f"\nAccuracy (zero-shot): {acc_zero:.4f}")

# One-shot
y_true_one, y_pred_one = run_mode("one")
acc_one = compute_accuracy(y_true_one, y_pred_one)
print(f"\nAccuracy (one-shot): {acc_one:.4f}")

# Few-shot
y_true_few, y_pred_few = run_mode("few")
acc_few = compute_accuracy(y_true_few, y_pred_few)
print(f"\nAccuracy (few-shot): {acc_few:.4f}")

# Вариант: вывести несколько примеров с предсказаниями
print("\nПримеры (test) с предсказаниями (true -> pred) для few-shot:")
for i in range(min(5, len(y_true_few))):
    print(f"{i+1}. [{y_true_few[i]}] → [{y_pred_few[i]}]  |  {test_ds[i]['text'][:60].strip()}...")


Запуск режима zero-shot на 500 примерах из test...


zero-shot classification: 100%|██████████| 500/500 [01:16<00:00,  6.52it/s]



Accuracy (zero-shot): 0.8080

Запуск режима one-shot на 500 примерах из test...


one-shot classification: 100%|██████████| 500/500 [01:24<00:00,  5.91it/s]



Accuracy (one-shot): 0.7820

Запуск режима few-shot на 500 примерах из test...


few-shot classification: 100%|██████████| 500/500 [01:31<00:00,  5.44it/s]


Accuracy (few-shot): 0.7180

Примеры (test) с предсказаниями (true -> pred) для few-shot:
1. [Business] → [Business]  |  Fears for T N pension after talks Unions representing worker...
2. [Sci/Tech] → [Science ---]  |  The Race is On: Second Private Team Sets Launch Date for Hum...
3. [Sci/Tech] → [Science ---]  |  Ky. Company Wins Grant to Study Peptides (AP) AP - A company...
4. [Sci/Tech] → [World]  |  Prediction Unit Helps Forecast Wildfires (AP) AP - It's bare...
5. [Sci/Tech] → [World]  |  Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern Ca...


# Выводы:

1. Видим, что наибольная accuracy получилась для zero-shot. Это происходит по нескольким причинам. При few-shot в промпте сразу несколько примеров, и итоговая длина может превышать ограничение. Из-за этого промт усекается, и модель не видит всех примеров, что снижает качество вывода. Когда в подсказке слишком много примеров, модель путается между ними и новым текстом, особенно в T5-small, где пропускная способность ограничена.

2. Заметим, что для категории Sci/Tech модель выдаёт Science. Это происходит потому, что модель не возвращает точные строки Sci/Tech, а генерирует ближайшее по смыслу слово (в этом случае Science). При попытке поиска точного совпадения с AG_LABELS Sci/Tech нижний регистр и формат не совпадают, поэтому такие ответы считаются некорректными. Но в реальности видно, что модель улавливает смысл новости.

3. T5-small не обладает достаточной емкостью для запоминания сразу нескольких примеров и сложных инструкций. Zero-shot промт короче, и модель точнее фокусируется на инструкции. Если примеры few-shot случайно содержат мало репрезентативных статей для каждой категории (например, три примера с большими перекосами), модель плохо обобщает на новые статьи.

4. Идейно, можно улучшить качество вот так:

   * Можно сократить кол-во примеров до 1–2 (one-shot или two-shot).
   * Можем явно указывать точный формат меток в описании задачи.
   * Возможно нужна более крупная модель, чтобы увеличить емкость и глубже обрабатывать контекст few-shot.
   * Еще мб помогло бы выбрать не случайные статьи, а какие-то тексты по всем 4 категориям.

5. Zero-shot промт короче и понятнее, поэтому T5-small лучше справляется без примеров. One-shot уже добавляет один образец, и производительность немного падает. Few-shot даёт слишком длинный, частично усечённый промт, что ухудшает качество.

6. То есть, для совсем небольших моделей есть смысл брать one- или two-shot, а для few-shot использовать модели с большим количеством параметров.
